In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm
import os
import urllib.parse 

# ======================
# CONFIGURACIÓN
# ======================
# NOTA: Asegúrate de que el archivo de entrada esté en el mismo directorio.

INPUT_FILE = "Punto1_NOMBREARCHIVORESULTADO.xlsx"                # ARCHIVO RESULTADO DEL SCRIPT 1
OUTPUT_FILE = "Punto2_NOMBREARCHIVORESULTADO.xlsx"               # ARCHIVO QUE RESULTADO DE ESTE SCRIPT 2

GITHUB_TOKEN = None        # AQUI PUEDES USAR TU TOKEN DE GitHub, ponlo aquí para mejores resultados
MAX_RESULTS = 4            # Número máximo de resultados a obtener por cada fuente (excepto enlaces de búsqueda)
SLEEP_TIME = 1             # Segundos de espera entre llamadas API para ser cortés

# ======================
# CROSSREF (papers/DOIs)
# ======================
def search_crossref(query, max_results=MAX_RESULTS):
    """Busca trabajos académicos en CrossRef usando el título del dataset."""
    results = []
    try:
        url = f"https://api.crossref.org/works?query.bibliographic={query}&rows={max_results}"
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            for item in r.json().get("message", {}).get("items", []):
                # CrossRef da las partes de la fecha. Usamos la primera parte (año, mes, día si existe)
                date_parts = item.get("issued", {}).get("date-parts", [[None]])[0]
                # Formato: ["YYYY", "MM", "DD"] -> "YYYY-MM-DD"
                date_str = "-".join(map(str, date_parts)) if date_parts and date_parts[0] else None 

                results.append({
                    "source_type": "crossref_paper",
                    "source_title": item.get("title", ["Sin título"])[0],
                    "year": date_str, # Usamos la fecha completa o YYYY-MM
                    "link": item.get("URL", "")
                })
    except Exception as e:
        print(f" Error CrossRef: {e}")
    return results

# ======================
#  OPENALEX (Investigación global)
# ======================
def search_openalex(query, max_results=MAX_RESULTS):
    """Busca trabajos de investigación en OpenAlex (la alternativa abierta a Scopus/WoS)."""
    results = []
    try:
        url = f"https://api.openalex.org/works?search={query}&per-page={max_results}"
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            for item in r.json().get("results", []):
                # OpenAlex devuelve la fecha de publicación completa
                year_full = item.get("publication_date", "") # Formato: YYYY-MM-DD
                year = year_full if year_full else None
                results.append({
                    "source_type": "openalex_work",
                    "source_title": item.get("title", "Sin título"),
                    "year": year, # Guardamos la fecha completa YYYY-MM-DD
                    "link": item.get("id", "").replace("https://openalex.org/", "https://openalex.org/works/") 
                })
    except Exception as e:
        print(f" Error OpenAlex: {e}")
    return results

# ======================
# ZENODO (Repositorio de datos y software)
# ======================
def search_zenodo(query, max_results=MAX_RESULTS):
    """Busca registros (datasets, software) en Zenodo."""
    results = []
    try:
        url = f"https://zenodo.org/api/records?q={query}&size={max_results}"
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            for item in r.json().get("hits", {}).get("hits", []):
                # Zenodo almacena la fecha de publicación completa
                year_full = item.get("metadata", {}).get("publication_date", "") # Formato: YYYY-MM-DD
                year = year_full if year_full else None
                
                results.append({
                    "source_type": "zenodo_record",
                    "source_title": item.get("metadata", {}).get("title", "Sin título"),
                    "year": year, # Guardamos la fecha completa YYYY-MM-DD
                    "link": item.get("links", {}).get("html", "")
                })
    except Exception as e:
        print(f" Error Zenodo: {e}")
    return results

# ======================
# GOOGLE SCHOLAR (Búsqueda de Enlace)
# ======================
def search_google_scholar(query, max_results=1):
    """Genera un enlace de búsqueda directa en Google Scholar."""
    q_encoded = urllib.parse.quote(query)
    results = [{
        "source_type": "google_scholar_search",
        "source_title": f"Búsqueda en Google Scholar: '{query[:40]}...'",
        "year": "N/A", # No hay fecha específica en un enlace de búsqueda
        "link": f"https://scholar.google.com/scholar?q={q_encoded}"
    }]
    return results[:max_results]

# ======================
# GITHUB
# ======================
def search_github(query, max_results=MAX_RESULTS, token=GITHUB_TOKEN):
    """Busca repositorios en GitHub."""
    results = []
    try:
        url = f"https://api.github.com/search/repositories?q={query}+in:name,description"
        headers = {"Accept": "application/vnd.github.v3+json"}
        if token:
            headers["Authorization"] = f"token {token}"
        r = requests.get(url, headers=headers, timeout=20)
        
        if r.status_code == 200:
            for item in r.json().get("items", [])[:max_results]:
                # GitHub proporciona el timestamp completo (YYYY-MM-DDTHH:MM:SSZ)
                updated_at = item.get("updated_at", "")[:10] 
                results.append({
                    "source_type": "github_repo",
                    "source_title": item.get("full_name", "Sin título"),
                    "year": updated_at, # Guardamos YYYY-MM-DD
                    "link": item.get("html_url", "")
                })
        elif r.status_code == 403:
            print("      Error GitHub (403): Límite de tasa excedido. Usa un GITHUB_TOKEN.")
        else:
            print(f"      Error GitHub ({r.status_code}): {r.text[:50]}...")

    except Exception as e:
        print(f" Error GitHub: {e}")
    return results

# ======================
# KAGGLE, APPS, PROYECTOS (Búsqueda de Enlace)
# ======================
# Estas funciones no devuelven fechas de reutilización específicas y siguen siendo enlaces genéricos.

def search_kaggle(query, max_results=2):
    """Genera un enlace de búsqueda directa en Kaggle."""
    results = [{
        "source_type": "kaggle_search",
        "source_title": f"Búsqueda en Kaggle: '{query[:50]}'",
        "year": "N/A", 
        "link": f"https://www.kaggle.com/search?q={urllib.parse.quote(query)}"
    }]
    return results[:max_results]

def search_apps(query, max_results=1):
    """Genera un enlace de búsqueda directa en Google Play Store (simulación)."""
    q_encoded = urllib.parse.quote(f"{query} datos andalucía")
    results = [{
        "source_type": "app_store_search",
        "source_title": f"Búsqueda de Apps: '{query[:40]}...'",
        "year": "N/A", 
        "link": f"https://play.google.com/store/search?q={q_encoded}&c=apps"
    }]
    return results[:max_results]

def search_projects(query, max_results=1):
    """Genera un enlace de búsqueda directa en el portal CORDIS (simulación)."""
    q_encoded = urllib.parse.quote(f"{query} andalucia")
    results = [{
        "source_type": "eu_project_search",
        "source_title": f"Búsqueda de Proyectos UE (CORDIS): '{query[:40]}...'",
        "year": "N/A", 
        "link": f"https://cordis.europa.eu/search?q={q_encoded}"
    }]
    return results[:max_results]

# ======================
# PIPELINE PRINCIPAL
# ======================
def run_reuse_search(input_file, output_file):
    # Carga del archivo de entrada
    try:
        # Asumimos Excel ya que es el formato original
        df = pd.read_excel(input_file) 
    except FileNotFoundError:
        print(f" Error: El archivo de entrada '{input_file}' no se encontró.")
        return pd.DataFrame() 

    results = []
    print(f" Analizando {len(df)} datasets del archivo {input_file}...")

    possible_title_cols = ["Titulo", "Título", "title", "dataset_title", "Nombre"]
    # Columna de fecha de publicación en el archivo de entrada
    published_date_col = "Fecha de publicación" 
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="🔎 Buscando reutilización"):
        
        dataset_id = row.get("Dataset_id", f"D{idx+1}") 

        # --- Buscar título válido ---
        dataset_title = None
        for col in possible_title_cols:
            if col in df.columns and pd.notna(row[col]):
                dataset_title = str(row[col])
                break
        
        if not dataset_title:
            dataset_title = "Sin título"
        
        # --- Extraer FECHA COMPLETA del dataset original ---
        # Si la columna existe y no es nula, toma el valor completo.
        # Esto captura el formato YYYY-MM-DDTHH:MM:SS de CKAN
        full_published_date = str(row.get(published_date_col, "")) if published_date_col in df.columns and pd.notna(row[published_date_col]) else ""
        
        # El año se sigue extrayendo para referencias rápidas (solo YYYY)
        year_dataset = full_published_date[:4] if full_published_date and len(full_published_date) >= 4 else ""

        # --- BUSCAR REUTILIZACIONES EN TODAS LAS FUENTES ---
        combined = []
        combined += search_crossref(dataset_title)
        combined += search_openalex(dataset_title)
        combined += search_zenodo(dataset_title)
        combined += search_google_scholar(dataset_title)
        combined += search_github(dataset_title, token=GITHUB_TOKEN)
        combined += search_kaggle(dataset_title)
        combined += search_apps(dataset_title)
        combined += search_projects(dataset_title)

        # --- Agregar contexto del dataset ---
        for hit in combined:
            hit["dataset_id"] = dataset_id
            hit["dataset_title"] = dataset_title
            # Conservamos el año de publicación del dataset (solo YYYY) 
            hit["year_dataset"] = year_dataset 
            # GUARDAMOS LA FECHA COMPLETA DEL DATASET ORIGINAL
            hit["published_date"] = full_published_date 
            results.append(hit)

        # Pausa para ser cortés con las APIs
        time.sleep(SLEEP_TIME)

    # --- Crear DataFrame final ---
    df_out = pd.DataFrame(results, columns=[
        "dataset_id", "dataset_title", "source_type", "source_title",
        "year", "year_dataset", "published_date", "link"
    ])

    df_out.to_excel(output_file, index=False)
    print(f"\n Resultados guardados en {output_file}")
    return df_out

# ======================
# EJECUTAR
# ======================
if __name__ == "__main__":
    df_results = run_reuse_search(INPUT_FILE, OUTPUT_FILE)
    if not df_results.empty:
        print("\n Muestra de resultados (10 primeras filas):")
        print(df_results.head(10).to_markdown(index=False))
